In [1]:
import pandas as np
import requests
import re
import click
import csv

In [2]:
class Book:
    def __init__(self, title, author, isbn, publisher, data_read, my_rating):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.my_rating = my_rating
        self.average_rating = ''
        self.publisher = publisher
        self.binding = ''
        self.year_published = ''
        self.original_publication_year = ''
        self.data_read = data_read
        self.data_added = ''
        self.bookshelves = ''
        self.my_review = ''

    def __str__(self):
        return f"{self.title} - {self.author}. (ISBN: {self.isbn})"

In [3]:
def fetch_book_isbn(book_url):
    isbn = 0
    r = requests.get(f"https://www.skoob.com.br{book_url}")

    if r.status_code == 200:
        match = re.search(r'<meta property="books:isbn" content="(\w+)"', r.text)
        if match:
            isbn = match.groups()[0]
    return isbn


def export_to_csv(user, books):
    with open(f'{user}-my-books.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Title', 'Author', 'ISBN', 'My Rating', 'Average Rating', 'Publisher', 'Binding',
                         'Year Published', 'Original Publication Year', 'Date Read', 'Date Added', 'Bookshelves',
                         'My Review'])
        for book in books:
            writer.writerow([book.title, book.author, book.isbn, book.my_rating, book.average_rating, book.publisher,
                             book.binding, book.year_published, book.original_publication_year, book.data_read,
                             book.data_added, book.bookshelves, book.my_review])

In [4]:
"""Skoob books exporter"""
books = []
page = 0
has_next = True
user=8132020
shelf=1 # 0 - Todos, 1 - Lidos, 2 - Lendo, 3 - Quero Ler, 4 - Relendo, 5 - Abandonei
book_json_2 = ""

print("Reading books from Skoob...")
while has_next:
    page += 1
    r = requests.get(f"https://www.skoob.com.br/v1/bookcase/books/{user}/shelf_id:{shelf}/page:{page}/limit:50/")

    if r.status_code == 200:
        json_response = r.json()
        books_response = json_response['response']
        has_next = json_response['paging']['next_page']
        for book_json in books_response:
            book_json_2 = book_json
            isbn = fetch_book_isbn(book_json['edicao']['url'])
            book = Book(book_json['edicao']['titulo'], book_json['edicao']['autor'].split(',')[0], isbn,
                        book_json['edicao']['editora'], book_json['dt_leitura'][:10], book_json['ranking'])
            books.append(book)
            print(book)

print("\nExporting books to csv...")
export_to_csv(user, books)
print("\nDone!")

Reading books from Skoob...
Le Petit Prince - Antoine de Saint-Exupéry. (ISBN: 2070256626)
ALEXANDER STONE - Cleo Luz. (ISBN: 0553380958)
Segredo lascivo - Natália Dias. (ISBN: 0553380958)
Só Entre Nós Dois - Fernanda Santana. (ISBN: 0553380958)
A Lenda de Ruff Ghanor - Leonel Caldela. (ISBN: 0553380958)
Loucos Por Livros - Emily Henry. (ISBN: 9786559241538)
A Boa Sorte - Álex Rovira Celma. (ISBN: 9788543103259)
Obcecado pela Protegida - D. A. Lemoyne. (ISBN: 0553380958)
O Juiz Bad Boy - Bianca Pohndorf. (ISBN: 0553380958)
Quando o conde encontra seu par - Stacy Reid. (ISBN: 0553380958)
Se não fosse por você - Laura Amorim. (ISBN: 0553380958)
A Paixão do Visconde - Stefany Nunes. (ISBN: 0553380958)
UM CASAMENTO FORÇADO - Ivy Matarazzo. (ISBN: 0553380958)
Essencialismo - Greg McKeown. (ISBN: 8543102146)
Pecados No Inverno - Lisa Kleypas. (ISBN: 0553380958)
Meu Grande Amor, Lucily - Julie Lopo. (ISBN: 0553380958)
Quem Mexeu no Meu Queijo? - Spencer Johnson. (ISBN: 9788501112064)
Abaixo d

In [6]:
books[0].isbn

'0553380958'

In [16]:
book_json_2

{'id': 187511396,
 'livro_id': 12227680,
 'ranking': 5,
 'tipo': 2,
 'favorito': 0,
 'desejado': 0,
 'troco': 0,
 'tenho': 0,
 'emprestei': 0,
 'paginas': 336,
 'dt_resenha': '',
 'paginas_lidas': 0,
 'dt_leitura': '2023-08-11 18:08:55',
 'meta': 0,
 'spoiler': 0,
 'media': 0,
 'percentual_lido': 0,
 'edicao': {'id': 12208890,
  'livro_id': 12227680,
  'titulo': 'A Razão do Amor',
  'nome_portugues': 'A Razão do Amor',
  'subtitulo': '',
  'subtitulo_portugues': '',
  'idioma': '',
  'mes': '',
  'ano': 2022,
  'volume': '',
  'serie': '',
  'autor': 'Ali Hazelwood',
  'isbn': 0,
  'paginas': 336,
  'edicao': 0,
  'editora': 'Arqueiro',
  'sinopse': 'No novo livro da autora best-seller de A hipótese de amor, uma cientista será obrigada a colaborar com seu rival em um projeto de proporções interestelares, e os resultados prometem ser explosivos.\r\n\r\n\x93Seu mundo está prestes a ser abalado.\x94 \x96 Elena Armas, autora de Uma farsa de amor na Espanha\r\n\r\nA carreira de Bee Königswa

In [8]:
with open('book.json', 'w') as f:
    f.write(str(book_json_2))

In [5]:
book_json_2

{'id': 219915315,
 'livro_id': 122227407,
 'ranking': 4,
 'tipo': 1,
 'favorito': 0,
 'desejado': 0,
 'troco': 0,
 'tenho': 0,
 'emprestei': 0,
 'paginas': 362,
 'dt_resenha': '',
 'paginas_lidas': 0,
 'dt_leitura': '2023-07-08 00:00:00',
 'meta': 0,
 'spoiler': 0,
 'media': 0,
 'edicao': {'id': 122233172,
  'livro_id': 122227407,
  'titulo': 'Se não fosse por você',
  'nome_portugues': 'Se não fosse por você',
  'subtitulo': '',
  'subtitulo_portugues': '',
  'idioma': '',
  'mes': '',
  'ano': 2023,
  'volume': '',
  'serie': '',
  'autor': 'Laura Amorim',
  'isbn': 0,
  'paginas': 368,
  'edicao': 0,
  'editora': 'Literare Books Internacional',
  'sinopse': 'Um romance que vai te deixar com vontade de correr atrás dos seus sonhos e, quem sabe, encontrar um grande amor no meio do caminho.\r\n\r\n"Se não fosse por você" é um presente para fãs de fake dating (namoro de mentira), mocinhos rabugentos e irritantemente bonitos, protagonistas fortes e o bom e velho slow burn!\r\n\r\nSe tem 